# method1 Data preprocessing(Spark-SQL)

In [22]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
sc = SparkContext.getOrCreate()
sqlContext = SQLContext.getOrCreate(sc)



## load rating data

In [23]:
ratings = sqlContext.read.format("com.databricks.spark.csv").\
option("header", "true").option("inferSchema", "true").\
load("file:///Users/qiankunhuang/Downloads/ml-latest-small/rating.csv")
ratings.createOrReplaceTempView("ratings")


+----+
|user|
+----+
| 671|
+----+



## calculate the number of user

In [39]:
count_user=spark.sql("select count(distinct(userId)) as user from ratings")
count_user.show()

+----+
|user|
+----+
| 671|
+----+



## load movies data

In [132]:
count_movie=sqlContext.read.format("com.databricks.spark.csv")\
    .option("header", "true").option("inferSchema", "true")\
    .load("file:///Users/qiankunhuang/Downloads/ml-latest-small/movies.csv")
count_movie.createOrReplaceTempView("movies")



## calculate the number of movies

In [134]:
count_movies=spark.sql("select count(distinct(movieId)) as movie from movies")
count_movies.show()

+-----+
|movie|
+-----+
| 9125|
+-----+



## calculate the number of rated_movies

In [33]:
count_movies1=spark.sql("select count(distinct(movieId)) as rated_movies \
                        from movies where movies.movieId \
                        in (select movieId from ratings) ")
count_movies1.show()



+------------+
|rated_movies|
+------------+
|        9066|
+------------+



## calculate the number of unrated_movies

In [35]:
count_movies2=spark.sql("select count(distinct(movieId)) as unrated_movies \
                       from movies where movies.movieId not in \
                       (select movieId from ratings) ")
count_movies2.show()


+--------------+
|unrated_movies|
+--------------+
|            59|
+--------------+



In [42]:
genres=spark.sql("select distinct(genres) as category from movies")
genres.show()


+--------------------+
|            category|
+--------------------+
|Comedy|Horror|Thr...|
|Adventure|Sci-Fi|...|
|Action|Adventure|...|
| Action|Drama|Horror|
|Comedy|Drama|Horr...|
|Action|Animation|...|
|Animation|Childre...|
|Action|Adventure|...|
| Adventure|Animation|
|    Adventure|Sci-Fi|
|Documentary|Music...|
|Adventure|Childre...|
|  Documentary|Sci-Fi|
| Musical|Romance|War|
|Action|Adventure|...|
|Adventure|Childre...|
|Crime|Drama|Fanta...|
|Comedy|Mystery|Th...|
|   Adventure|Fantasy|
|Action|Animation|...|
+--------------------+
only showing top 20 rows



## calculate the number of movies for category

In [41]:
count_movies=spark.sql("select count(distinct(movieId)) as movie_id,genres from movies group by genres")
count_movies.show()

+--------+--------------------+
|movie_id|              genres|
+--------+--------------------+
|       3|Adventure|Sci-Fi|...|
|      13|Comedy|Horror|Thr...|
|       5|Action|Adventure|...|
|       1|Comedy|Drama|Horr...|
|       1| Action|Drama|Horror|
|       1|Action|Animation|...|
|      26|Action|Adventure|...|
|       1| Adventure|Animation|
|      13|    Adventure|Sci-Fi|
|       1|Adventure|Childre...|
|       1|  Documentary|Sci-Fi|
|       2|Documentary|Music...|
|       2|Animation|Childre...|
|       1|Action|Adventure|...|
|       1|Crime|Drama|Fanta...|
|       2|Adventure|Childre...|
|       1| Musical|Romance|War|
|       3|Comedy|Mystery|Th...|
|      12|   Adventure|Fantasy|
|       1|Action|Animation|...|
+--------+--------------------+
only showing top 20 rows



# method2 Data preprocessing(RDD,DataFrame)


## load rating data

In [105]:
import pyspark
sc = SparkContext.getOrCreate()

In [127]:
ratings = sc.textFile("file:///Users/qiankunhuang/Downloads/ml-latest-small/rating.csv")
rating=ratings.map(lambda line:line.split(","))
movies= sc.textFile("file:///Users/qiankunhuang/Downloads/ml-latest-small/movies.csv")
movies= movies.map(lambda line:line.split(","))


In [129]:
head1= rating.first()
rating = rating.filter(lambda x : x!= head1)
head2= movies.first()
movies2= movies.filter(lambda x : x!= head2)



## calculate the number of user

In [121]:
users = rating.map(lambda x:(x[0],1)).reduceByKey(lambda a,b:a).count()
users


671

## calculate the number of movies

In [131]:
movies3 = movies2.map(lambda x:(x[0],1)).reduceByKey(lambda a,b:a).count()
movies3



9125